# RAG with multi-source data - retrieving information about client acquisition

## Tool 1: From wikipedia inbuilt tool

In [1]:
from langchain_community.tools import WikipediaQueryRun   # Responsible to fect the relevent result from the API using a query
from langchain_community.utilities import WikipediaAPIWrapper #responsible for communicating with the API

In [2]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)
help(wiki)

Help on WikipediaQueryRun in module langchain_community.tools.wikipedia.tool object:

class WikipediaQueryRun(langchain_core.tools.base.BaseTool)
 |  WikipediaQueryRun(
 |      *,
 |      name: str = 'wikipedia',
 |      description: str = 'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.',
 |      args_schema: Type[pydantic.main.BaseModel] = <class 'langchain_community.tools.wikipedia.tool.WikipediaQueryInput'>,
 |      return_direct: bool = False,
 |      verbose: bool = False,
 |      callbacks: Union[list[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None,
 |      callback_manager: Optional[langchain_core.callbacks.base.BaseCallbackManager] = None,
 |      tags: Optional[list[str]] = None,
 |      metadata: Optional[dict[str, Any]] = None,
 |      handle_tool_error: Union[bo

In [3]:
from langchain.document_loaders import WebBaseLoader # for loading the data from the website
from langchain.text_splitter import RecursiveCharacterTextSplitter # for splitting the data into chunks
from langchain_ollama import OllamaEmbeddings # For embedding the chunks into vectors
from langchain_community.vectorstores import FAISS, Pinecone, Weaviate # For storing the vectors
from langchain_ollama import OllamaLLM

# Unlike the previous RAG, we are not using a retriever chain from langchain.chains import create_retriever_chain
# Instead we are using a tool/agent from langchain.tools.retriever import create_retrieval_tool

USER_AGENT environment variable not set, consider setting it to identify your requests.


## Tool 2: From a website about client acquisition

In [4]:
# Using all the modules above to retrieve all the sources for RAG architecture
loader=WebBaseLoader("https://www.copilot.com/blog/client-acquisition")
docs= loader.load()
splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splited_doc=splitter.split_documents(docs)
db=FAISS.from_documents(splited_doc,OllamaEmbeddings(model="deepseek-r1:1.5b"))
retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x115bb30e0>, search_kwargs={})

In [5]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, "Name: Retriever Tool", "Description: Search for client acquisition")
retriever_tool

Tool(name='Name: Retriever Tool', description='Description: Search for client acquisition', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x11439e8e0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x115bb30e0>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x1148ccfe0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x115bb30e0>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', respo

## Tool 3: Arxiv as a tool

In [6]:
from langchain_community.tools import ArxivQueryRun   # Responsible for communicating with the API
from langchain_community.utilities import ArxivAPIWrapper # Responsible to fetch the relevent result from the API using a query


In [7]:
arxiv_wrapper= ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv

ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250))

## Combining all sources and making a RAG using agents


In [8]:
tools=[retriever_tool, arxiv, wiki]
tools

[Tool(name='Name: Retriever Tool', description='Description: Search for client acquisition', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x11439e8e0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x115bb30e0>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x1148ccfe0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x115bb30e0>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', resp

In [9]:
from dotenv import load_dotenv
import os
load_dotenv()

os.environ['LANGSMITH_API_KEY']=os.getenv('LANGSMITH_API_KEY')
os.environ['LANGSMITH_TRACING'] = os.getenv('LANGSMITH_TRACING')

#loading the LLM
llm=OllamaLLM(model="deepseek-r1:1.5b")
llm

OllamaLLM(model='deepseek-r1:1.5b')

In [10]:
#get the prompt from hub
# from langchain import hub 
# prompt=hub.pull("hardkothari/prompt-maker")
# prompt
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("user", "{question}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [11]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant.'), additional_kwargs={}),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

## Making Agents

In [12]:
# Make an agent
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm, tools, prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'question'], input_types={'agent_scratchpad': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(ta

In [13]:
# Making an agent executor to execute the agent
from langchain.agents import AgentExecutor
agent_executor= AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'question'], input_types={'agent_scratchpad': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], ty

In [15]:
# You can now call the agent via agent executor and give it prompt
from transformers import RagModel, RagTokenizer

model = RagModel.from_pretrained("facebook/rag-token-nq")
output = model.generate("What are the 10 strategies for client acquisition?", tools=tools)
output


# agent_executor.invoke({'question': "What are the 10 strategies for client acquisition?"})


ImportError: 
RagModel requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.
